In [6]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import sklearn
import numpy as np

df = pd.read_json("sentenceLengthTrain.jsonl", lines=True)
dev = pd.read_json("sentenceLengthDev.jsonl", lines=True)

labels = None
with open("train-labels.lst") as l:
    labels = l.readlines()
    labels = [int(int(line.rstrip()) == 2) for line in labels]

train_labels = pd.DataFrame(labels, columns =['label'])

with open("dev-labels.lst") as l:
    labels = l.readlines()
    labels = [int(int(line.rstrip()) == 2) for line in labels]

dev_labels = pd.DataFrame(labels, columns =['label'])



sorted_inds = np.argsort(df.sent_length)
df = df.iloc[sorted_inds]
train_labels = train_labels.iloc[sorted_inds]


sorted_inds = np.argsort(dev.sent_length)
dev = dev.iloc[sorted_inds]
dev_labels = dev_labels.iloc[sorted_inds]

In [7]:
df = pd.DataFrame({
        'text_a': '[CLS] ' + df['obs1'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + df['obs2'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + df['hyp1'].replace(r'\n', ' ', regex= True),
        'text_b': '[CLS] ' + df['obs1'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + df['obs2'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + df['hyp2'].replace(r'\n', ' ', regex = True),
        'labels': train_labels['label'],
        'length': df['sent_length']
    })


dev = pd.DataFrame({ 
        'text_a': '[CLS] ' + dev['obs1'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + dev['obs2'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + dev['hyp1'].replace(r'\n', ' ', regex= True),
        'text_b': '[CLS] ' + dev['obs1'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + dev['obs2'].replace(r'\n', ' ', regex= True) + ' [SEP] ' + dev['hyp2'].replace(r'\n', ' ', regex = True),
        'labels': dev_labels['label'],
        
    })

In [8]:
df_1 = df[(df['length'] < 0.25)]
df_2 = df[(df['length'] < 0.5)]
df_3 = df[(df['length'] < 0.75)]
df_4 = df

df = [df_1.drop(columns='length'), df_2.drop(columns='length'), df_3, df_4]

In [9]:
print("Bin 1: " + df_1.shape)
print("Bin 2: " + df_2.shape)
print("Bin 3: " + df_3.shape)
print("Bin 4: " + df_4.shape)

(40542, 4)
(78132, 4)
(123673, 4)
(169654, 4)


In [19]:
import os  
num_bins = 4
model_path = 'outputs/curriculum_anli'

for i in range(0, num_bins):
    if os.path.isdir('./outputs/curriculum_anli'):
        print("Found the model! Continuing training with bin " + str(i+1))
    else:
        print("Could not find the model! Starting training with bin " + str(i+1))
        model_path = 'roberta-large'
        
    model = ClassificationModel('roberta', model_path, 
                                args={'overwrite_output_dir': True,
                                         'output_dir': 'outputs/curriculum_anli',
                                         'fp16': False,
                                         'num_train_epochs': 1,  
                                         'reprocess_input_data': False,
                                         "learning_rate": 5e-6,                                       
                                         "train_batch_size": 16,
                                         "eval_batch_size": 16,
                                         "weight_decay": 0.2,
                                         "max_seq_length": 128,
                                         "evaluate_during_training_verbose": True,
                                         "evaluate_during_training": True,
                                         "weight_decay": 0,
                                         "do_lower_case": True,
                                         "n_gpu": 4, # can be 1 if you have enough memory
                                     })

                                  
    # Train the model
    model.train_model(df[i], eval_df=dev)
    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(dev, acc=sklearn.metrics.accuracy_score)
    print(result)



Could not find the model! Starting training with bin 1


Running loss: 0.021599




{'mcc': 0.4898184680486515, 'tp': 566, 'tn': 575, 'fp': 206, 'fn': 185, 'acc': 0.7447780678851175, 'eval_loss': 0.8236465589919438}
Found the model! Continuing training with bin 2


Running loss: 0.200955




{'mcc': 0.5403808067598695, 'tp': 578, 'tn': 602, 'fp': 179, 'fn': 173, 'acc': 0.7702349869451697, 'eval_loss': 0.8602239430183545}
Found the model! Continuing training with bin 3


Running loss: 0.002346




{'mcc': 0.5758284310854256, 'tp': 596, 'tn': 611, 'fp': 170, 'fn': 155, 'acc': 0.7878590078328982, 'eval_loss': 0.8448484289304664}
Found the model! Continuing training with bin 4


Running loss: 0.337460




{'mcc': 0.6094663958310761, 'tp': 598, 'tn': 635, 'fp': 146, 'fn': 153, 'acc': 0.8048302872062664, 'eval_loss': 0.8322111684828997}
